# Exercise: Implementing a Sudoku solver and testing for uniqueness

<b>Goal:</b> In this exercise, we implement a Sudoku solver using an integer program and Python-MIP. As an extra result, we will also see how such a solver can be used to test uniqueness of a solution to a given Sudoku puzzle.

---

## Solving a Sudoku puzzle: Modeling with integer variables

As you most likely all know, a Sudoku is puzzle where the goal is to fill in the cells of a 9x9 grid with integers from $1$ to $9$ such that
- there is precisely one number per cell,
- no row contains two equal numbers,
- no column contains two equal numbers, and
- no one of the nine 3x3 squares that the grid can be partitioned in contains two equal numbers.

An example of a Sudoku puzzle is given below.

<div style="background-color:white">
<center>
    <img src="sudoku_solver_example.png", style="padding-top: 10px;">
</center>
</div>

To solve such a Sudoku puzzle, we have to decide which numbers to assign to which cell. These decisions can be easily modelled by integral decision variables $x_{ijk}$ for $i,j,k\in\{1,\ldots,9\}$ such that

$$
x_{ijk} = \begin{cases} 1 & \text{if cell $(i,j)$ of the Sudoku contains number $k$}\\ 0 & \text{else} \end{cases}\enspace. 
$$

The question is how to set up suitable constraints that guarantee that a feasible $\{0,1\}$-point $x$ does indeed correspond to a solution of the Sudoku.


<b>Your first task:</b> Come up with linear constraints in the variables $x_{ijk}$ that model the conditions imposed on a valid Sudoku solution, i.e., make sure that any $\{0,1\}$-solution of your system corresponds to a feasible solution of a given Sudoku.

$$
\sum_{i\in{1,...,9}}^{}{x_{ijk}} = 1 \quad\forall k, k \\
\sum_{j\in{1,...,9}}^{}{x_{ijk}} = 1 \quad\forall i, k\\
\sum_{k\in{1,...,9}}^{}{x_{ijk}} = 1 \quad\forall i, j \\
\sum_{i \in \{ i \in \{1,...,9\} | i < 3f \wedge i > 3(f-1) \}}^{}{\sum_{j \in \{ j \in \{1,...,9\} | j <= 3g \wedge j > 3(g-1)\}}}{x_{ijk}} = 1 \quad \forall k,\forall f \in \{1,2,3\} , \forall g \in \{1,2,3\}
$$

---

## Implementing integer programs in Python-MIP

Implementing integer programs in Python-MIP is almost the same as implementing linear programs - except that you'll have to declare that you want to put integrality conditions on your variables. Check out the simple IP below.

In [491]:
import mip

simpleProblem = mip.Model(name="Simple IP example", sense=mip.MINIMIZE)

x = simpleProblem.add_var(name="x", var_type=mip.INTEGER)

simpleProblem.objective = x

simpleProblem += x >= 4.5

simpleProblem.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [492]:
print(x.x)

5


As you can see, the `add_var` takes an additional (optional) argument `var_type`, which we can set to `INTEGER`, `BINARY` or `CONTINUOUS` (the latter one being the default). Thus, for the Sudoku problem above, you might want to use binary variables.

<b>Your second task:</b> Implement the constraints that you came up with in the first task in an integer program, and use it to find a solution to an input Sudoku problem. Observe that this is a pure feasibility problem, so you can use an IP with a constant objective.

To this end, you can assume that the Sudoku is given to you as a list of $81$ values, each representing a cell of the Sudoku read row by row from left to right; where a $0$ indicates an empty cell. An example is given below. Note that there also is a function to display a Sudoku that is given in the above form.

Make sure that your function returns the Sudoku in the same format as the input.

In [493]:
# Example Sudoku input and Sudoku printing

sudoku1 = [4, 0, 7, 0, 0, 0, 0, 0, 0, 
           0, 3, 5, 0, 9, 7, 4, 0, 0, 
           0, 9, 0, 0, 0, 0, 0, 0, 6, 
           0, 0, 0, 3, 0, 2, 0, 0, 0, 
           6, 0, 0, 0, 8, 0, 0, 0, 0, 
           0, 0, 0, 0, 0, 0, 5, 0, 0, 
           0, 0, 0, 4, 0, 0, 0, 1, 8, 
           0, 0, 3, 0, 2, 8, 0, 0, 4, 
           5, 0, 4, 0, 0, 0, 0, 9, 7]

def printSudoku(sudoku):
    # compact Sudoku printing function
    # taken from https://codegolf.stackexchange.com/questions/126930/
    #    draw-a-sudoku-board-using-line-drawing-characters
    q = lambda x,y:x+y+x+y+x
    r = lambda a,b,c,d,e:a+q(q(b*3,c),d)+e+"\n"
    print(((r(*"╔═╤╦╗") + q(q("║ %d │ %d │ %d "*3 + "║\n",r(*"╟─┼╫╢")), r(*"╠═╪╬╣")) +
            r(*"╚═╧╩╝")) % tuple(sudoku)).replace(*"0 "))

printSudoku(sudoku1)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 4 │   │ 7 ║   │   │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 3 │ 5 ║   │ 9 │ 7 ║ 4 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 9 │   ║   │   │   ║   │   │ 6 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║ 3 │   │ 2 ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │   │   ║   │ 8 │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │   │   ║ 5 │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║ 4 │   │   ║   │ 1 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 3 ║   │ 2 │ 8 ║   │   │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │   │ 4 ║   │   │   ║   │ 9 │ 7 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝



In [494]:
# Implementation of a Sudoku solver
import math

def sudokuSolver(inputSudoku, nonSols = []):

    # to solve any sudoku, even bigger / smaller ones
    n = int(math.sqrt(len(inputSudoku)))
    sudokumodel = mip.Model(name="sudoku")

    x_vars = []

    for i in range(n):
        x_vars.append([])
        for j in range(n):
            x_vars[i].append([])
            for k in range(n):
                x_vars[i][j].append([])
                x_vars[i][j][k] = sudokumodel.add_var(name=f"x_{i}_{j}_{k}", var_type=mip.BINARY)

    for i in range(n):
        for j in range(n):
            sudokumodel += mip.xsum(x_vars[i][j][k] for k in range(n)) == 1
    for i in range(n):
        for k in range(n):
            sudokumodel += mip.xsum(x_vars[i][j][k] for j in range(n)) == 1
    for j in range(n):
        for k in range(n):
            sudokumodel += mip.xsum(x_vars[i][j][k] for i in range(n)) == 1


    squaresPerRow = int(math.sqrt(n))
    for k in range(n):
        for f in range(squaresPerRow):
            for g in range(squaresPerRow):
                sudokumodel+= mip.xsum(x_vars[i][j][k]
                                       for i in range(n) if (i < (f + 1) * squaresPerRow and i >= f * squaresPerRow)
                                       for j in range(n) if (j < (g + 1) * squaresPerRow and j >= g * squaresPerRow)) == 1

    # input sudoku
    for i in range(n):
        for j in range(n):
            inputVal = inputSudoku[j + (i * n)]
            if (inputVal > 0):
                sudokumodel += x_vars[i][j][inputVal - 1] >= 1

    # nonsols
    for nonSol in nonSols:
        # print([f"{nonSol[j + (i*n)] - 1} in {i} {j}" for i in range(n) for j in range(n)])
        sudokumodel += mip.xsum(x_vars[i][j][nonSol[j + (i*n)] - 1] for i in range(n) for j in range(n)) <= 80

    sudokumodel.objective = 0
    result = sudokumodel.optimize()
    if (result == mip.OptimizationStatus.INFEASIBLE):
        return []

    outputSudoku = []
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if (x_vars[i][j][k].x == 1):
                    outputSudoku.append(k + 1)

    return outputSudoku

---

## Checking for uniqueness of the Sudoku solutions

Sudokus are generally agreed to only be "real" Sudokus if they have a unique Solution.

<b>Your third task:</b> Implement a function that checks whether a Sudoko has no solution, a unique solution, or more than one solution! You can reuse the code that you generated for the Sudoku solver above. The function should return a tuple `(n, sol)`, where $n\in\{0, 1, 2\}$ depending on whether the Sudoku has zero, one, or at least two solutions, respectively, and `sol` is a list of zero, one, or two solutions of the Sudoku.

If you want a hint, run the following code cell. Do not run it if you want to think about the problem yourself! :)

In [495]:
## Running this cell will display a hint!

encoded = [79, 98, 115, 101, 114, 118, 101, 32, 116, 104, 97, 116, 32, 115, 111, 108, 118, 105, 110, 103, 32, 97, 32, 83, 117, 100, 111, 107, 117, 32, 100, 105, 100, 32, 110, 111, 116, 32, 117, 115, 101, 32, 116, 104, 101, 32, 111, 98, 106, 101, 99, 116, 105, 118, 101, 32, 102, 117, 110, 99, 116, 105, 111, 110, 32, 111, 102, 32, 116, 104, 101, 32, 73, 80, 46, 32, 79, 110, 99, 101, 32, 121, 111, 117, 32, 102, 111, 117, 110, 100, 32, 111, 110, 101, 32, 115, 111, 108, 117, 116, 105, 111, 110, 44, 32, 99, 97, 110, 32, 121, 111, 117, 32, 101, 120, 112, 108, 111, 105, 116, 32, 116, 104, 101, 32, 102, 97, 99, 116, 32, 116, 104, 97, 116, 32, 121, 111, 117, 32, 99, 97, 110, 32, 99, 104, 111, 111, 115, 101, 32, 116, 104, 101, 32, 111, 98, 106, 101, 99, 116, 105, 118, 101, 32, 116, 111, 32, 115, 101, 101, 32, 105, 102, 32, 121, 111, 117, 32, 99, 97, 110, 32, 102, 105, 110, 100, 32, 97, 110, 111, 116, 104, 101, 114, 32, 115, 111, 108, 117, 116, 105, 111, 110, 63]
print('Hint: ' + ''.join([chr(x) for x in encoded]))

Hint: Observe that solving a Sudoku did not use the objective function of the IP. Once you found one solution, can you exploit the fact that you can choose the objective to see if you can find another solution?


In [496]:
# we can exploit the objective function but an easier solution (in my opinion) is simply to add a constraint to prevent the optained solution

def numberOfSolutions(inputSudoku):
    sols = []
    n = 0
    while True:
        sol = sudokuSolver(inputSudoku, sols)
        if (len(sol) == 0):
            break
        n += 1
        sols.append(sol)

    return (n, sols)

---

## Testing your code

Among the following three Sudokus, there is one from each category that your function `numberOfSolutions()` should be able to distinguish: One has no solution, one has a unique Solution, and one has two Solutions. Test your implementation on these Sudokus!

In [497]:
sudoku2 = [2, 0, 0, 0, 0, 0, 0, 4, 0, 
           1, 0, 0, 0, 0, 0, 0, 0, 7,
           8, 0, 6, 3, 0, 0, 0, 0, 0,
           0, 5, 0, 0, 0, 7, 3, 0, 1, 
           0, 0, 3, 0, 1, 0, 0, 0, 0, 
           0, 0, 2, 0, 0, 3, 7, 5, 4, 
           0, 0, 7, 0, 0, 5, 0, 0, 0, 
           5, 0, 0, 0, 4, 0, 0, 0, 0, 
           0, 0, 0, 1, 7, 0, 0, 0, 8]
printSudoku(sudoku2)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 2 │   │   ║   │   │   ║   │ 4 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │   │   ║   │   │   ║   │   │ 7 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 8 │   │ 6 ║ 3 │   │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │ 5 │   ║   │   │ 7 ║ 3 │   │ 1 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 3 ║   │ 1 │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 2 ║   │   │ 3 ║ 7 │ 5 │ 4 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │ 7 ║   │   │ 5 ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │   │   ║   │ 4 │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║ 1 │ 7 │   ║   │   │ 8 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝



In [498]:
sudoku3 = [0, 0, 0, 6, 0, 7, 0, 0, 0, 
           0, 0, 0, 0, 0, 0, 0, 9, 8,
           3, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 2, 0, 6, 0, 0, 
           0, 0, 0, 0, 0, 0, 7, 0, 0, 
           0, 4, 0, 0, 8, 0, 0, 0, 0, 
           1, 0, 0, 0, 0, 0, 0, 2, 3, 
           0, 0, 8, 9, 0, 0, 0, 0, 0, 
           0, 0, 0, 4, 0, 0, 1, 0, 0]
printSudoku(sudoku3)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║   │   │   ║ 6 │   │ 7 ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │   │   ║   │ 9 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │   │   ║   │   │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │ 2 │   ║ 6 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │   │   ║ 7 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 4 │   ║   │ 8 │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 1 │   │   ║   │   │   ║   │ 2 │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 8 ║ 9 │   │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║ 4 │   │   ║ 1 │   │   ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝



In [499]:
sudoku4 = [0, 6, 0, 0, 0, 0, 0, 7, 4,
           1, 0, 0, 6, 0, 7, 0, 0, 3, 
           7, 0, 0, 0, 0, 0, 0, 0, 0, 
           0, 0, 0, 0, 1, 0, 0, 0, 2, 
           0, 0, 1, 5, 0, 0, 9, 0, 0, 
           9, 0, 0, 8, 0, 0, 0, 1, 0, 
           0, 0, 0, 0, 0, 0, 0, 3, 0, 
           3, 0, 0, 0, 0, 2, 8, 5, 0, 
           0, 0, 9, 0, 0, 4, 0, 0, 0]
printSudoku(sudoku4)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║   │ 6 │   ║   │   │   ║   │ 7 │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │   │   ║ 6 │   │ 7 ║   │   │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │   │   ║   │   │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │ 1 │   ║   │   │ 2 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 1 ║ 5 │   │   ║ 9 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │   │   ║ 8 │   │   ║   │ 1 │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │   │   ║   │ 3 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │   │   ║   │   │ 2 ║ 8 │ 5 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 9 ║   │   │ 4 ║   │   │   ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝



In [ ]:
# Test your functions here!

def solveSudoku(sudoku):
    sudokuSol = numberOfSolutions(sudoku)
    print(f"found {sudokuSol[0]} solutions")
    for i in range(sudokuSol[0]):
        printSudoku(sudokuSol[1][i])
solveSudoku(sudoku2)
solveSudoku(sudoku3)
solveSudoku(sudoku4)
# printSudoku(sudokuSolver(sudoku2))

found 0 solutions
found 1 solutions
╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 4 │ 8 │ 2 ║ 6 │ 9 │ 7 ║ 3 │ 1 │ 5 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │ 6 │ 1 ║ 2 │ 5 │ 3 ║ 4 │ 9 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │ 5 │ 9 ║ 8 │ 4 │ 1 ║ 2 │ 7 │ 6 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 8 │ 7 │ 3 ║ 1 │ 2 │ 9 ║ 6 │ 5 │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │ 1 │ 5 ║ 3 │ 6 │ 4 ║ 7 │ 8 │ 2 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 2 │ 4 │ 6 ║ 7 │ 8 │ 5 ║ 9 │ 3 │ 1 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 1 │ 9 │ 4 ║ 5 │ 7 │ 6 ║ 8 │ 2 │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │ 3 │ 8 ║ 9 │ 1 │ 2 ║ 5 │ 4 │ 7 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │ 2 │ 7 ║ 4 │ 3 │ 8 ║ 1 │ 6 │ 9 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝

